Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots,ApproxFun

In [3]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [4]:
T = π;a=0.15
t = Fun(identity,[0,T])
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

Φ = A\eye(n);

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [5]:
ΦT = Φ(T)

4x4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [6]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.6653345369377348e-16 - 0.2683546905319053im
-2.6075062248276382e-17 + 0.2683546905319057im
0.03747531973228921 + 0.9999999999999994im
-0.03747531973228907 + 1.0000000000000002im

Floquet Multipliers = 

0.6651802570066602 - 0.746682814646589im
0.6651802570066595 + 0.7466828146465901im
-1.1249427991479368 + 2.136065595020538e-15im
-0.888934086921956 - 6.806690441197217e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [7]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

208

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [9]:
t = Fun(identity,[0,10T])
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

[Plots.jl] Initializing backend: gr


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,162.193 35.4377,162.167 35.4471,162.116 35.4611,162.039 35.4797,161.937 35.5031,161.809 35.5311,161.655 35.5637,161.475 35.601,161.27 35.643,161.038 
 35.6897,160.781 35.741,160.497 35.797,160.187 35.8576,159.851 35.9229,159.488 35.9928,159.099 36.0674,158.683 36.1467,158.24 36.2306,157.77 36.3192,157.273 
 36.4124,156.749 36.5103,156.197 36.6128,155.617 36.72,155.01 36.8318,154.374 36.9482,153.711 37.0693,153.019 37.1951,152.298 37.3254,151.548 37.4604,150.77 
 37.6001,149.963 37.7443,149.126 37.8932,148.259 38.0467,147.363 38.2049,146.438 38.3677,145.482 38.535,144.496 38.707,143.48 38.8836,142.433 39.0649,141.356 
 39.2507,140.248 39.4411,139.11 39.6361,137.941 39.8357,136.741 40.04,135.51 40.2488,134.249 40.4621,132.956 40.6801,131.633 40.9027,130.28 41.1298,128.896 
 41.3615,127.482 41.5977,126.037 41.8386,124.563 42.084,123.059 42.3339,121.527 42.5884,119.965 42.8474,118.375 43.111,116.757 43.3791,115.112 43.6518,113.44 
 43.929,111.743 44.2107,110.02 44.4969,108.273 44.7877,106.502 45.0829,104.709 45.3827,102.895 45.687,101.06 45.9957,99.2061 46.309,97.3347 46.6267,95.447 
 46.9489,93.5446 47.2756,91.629 47.6067,89.702 47.9424,87.7654 48.2824,85.8211 48.627,83.8711 48.976,81.9176 49.3294,79.9628 49.6872,78.0091 50.0495,76.0589 
 50.4162,74.1149 50.7873,72.1797 51.1629,70.2563 51.5428,68.3474 51.9271,66.4563 52.3159,64.5861 52.709,62.7401 53.1065,60.9218 53.5083,59.1347 53.9145,57.3824 
 54.3251,55.6689 54.7401,53.998 55.1594,52.3737 55.583,50.8001 56.0109,49.2817 56.4432,47.8226 56.8798,46.4274 57.3207,45.1006 57.7659,43.847 58.2154,42.6712 
 58.6692,41.5783 59.1272,40.573 59.5896,39.6605 60.0562,38.8458 60.527,38.134 61.0021,37.5305 61.4815,37.0404 61.965,36.669 62.4528,36.4217 62.9449,36.3038 
 63.4411,36.3207 63.9415,36.4776 64.4461,36.7799 64.9549,37.2328 65.4679,37.8415 65.9851,38.6111 66.5064,39.5466 67.0318,40.6529 67.5614,41.9348 68.0951,43.3967 
 68.633,45.0431 69.1749,46.8781 69.721,48.9055 70.2711,51.129 70.8254,53.5518 71.3837,56.1769 71.9461,59.0066 72.5125,62.0432 73.083,65.2883 73.6575,68.7429 
 74.2361,72.4076 74.8187,76.2823 75.4053,80.3666 75.9959,84.6589 76.5905,89.1573 77.189,93.8591 77.7916,98.7606 78.3981,103.858 79.0085,109.145 79.6229,114.616 
 80.2412,120.264 80.8635,126.082 81.4896,132.06 82.1197,138.189 82.7536,144.457 83.3914,150.854 84.0331,157.366 84.6787,163.98 85.3281,170.681 85.9813,177.455 
 86.6384,184.284 87.2992,191.153 87.9639,198.042 88.6323,204.935 89.3046,211.811 89.9806,218.653 90.6604,225.438 91.3439,232.148 92.0311,238.762 92.7221,245.26 
 93.4168,251.621 94.1152,257.824 94.8173,263.85 95.523,269.678 96.2324,275.29 96.9455,280.666 97.6622,285.789 98.3826,290.642 99.1065,295.207 99.8341,299.471 
 100.565,303.419 101.3,307.038 102.038,310.317 102.78,313.245 103.526,315.814 104.275,318.017 105.027,319.847 105.783,321.301 106.543,322.376 107.305,323.07 
 108.072,323.385 108.842,323.322 109.615,322.884 110.392,322.076 111.172,320.904 111.956,319.377 112.743,317.502 113.533,315.29 114.327,312.752 115.124,309.901 
 115.924,306.749 116.728,303.311 117.535,299.602 118.345,295.638 119.159,291.435 119.976,287.012 120.796,282.385 121.62,277.573 122.446,272.595 123.276,267.469 
 124.109,262.216 124.945,256.856 125.785,251.407 126.627,245.89 127.473,240.324 128.322,234.731 129.174,229.129 130.029,223.537 130.887,217.976 131.749,212.464 
 132.613,207.019 133.48,201.658 134.351,196.399 135.224,191.258 136.101,186.25 136.98,181.389 137.862,176.687 138.748,172.157 139.636,167.81 140.527,163.652 
 141.422,159.693 142.319,155.937 143.219,152.388 144.121,149.049 145.027,145.92 145.936,143 146.847,140.286 147.761,137.772 148.678,135.452 149.598,133.319 
 150.52,131.362 151.446,129.571 15